In [ ]:
import os
import sys
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import GithubFileLoader
from langchain_milvus import Milvus
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from sentence_transformers import SentenceTransformer
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, ToolCallingAgent
from IPython.display import Image, display

# Add parent directory to path to import sandbox modules
sys.path.append(str(Path.cwd().parent))
from sandbox_server import SandboxServer
from sandbox_client import SandboxClient

In [ ]:
endpoint = os.getenv("PHOENIX_TRACE_ENDPOINT", "http://0.0.0.0:6006/v1/traces")
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
# Standard HF Model
hf_model = HfApiModel(
    model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
)

In [ ]:
# Use the client with async context manager
async with SandboxClient("ws://localhost:8765") as client:
    # Test screenshot functionality
    result = await client.get_screenshot_with_metadata()

    if "screenshot_path" in result:
        print(f"Mouse position: {result['mouse_position']}")
        print(f"Screen size: {result['screen_size']}")
        display(Image(filename=result["screenshot_path"]))

In [ ]:
# agent = CodeAgent(
#     tools=[go_back, close_popups, search_item_ctrl_f],
#     model=hf_model,
#     additional_authorized_imports=["helium"],
#     step_callbacks=[save_screenshot],
#     max_steps=20,
#     verbosity_level=2,
# )
